# Lecture 3 - Discrete random variables, distributions and their moments

## L3.1 Random variables

A **random variable** is a function from the sample space to real numbers, i.e., $X:\Omega->\mathbb{R}$

A random variable is *discrete* if its range  (i.e., the set of values it can take) is either finite or countibly infinite. 

Examples of discrete random variables are the outcomes of a throw of a die, or sum of the throw of two dice. These have finite range, but an example of countably infinite range is the number of throws of a dice before you get a 6. The outcome of a draw of card from a deck is not a random variable (the outcome is not a real number).

Random variables that can take uncountably infinite number of values is not discrete. 

Examples of such variables are choosing a point from the interval [0,1] and the height of a randomly chosen person, or temperature tomorrow, etc.

Random variables all have distributions and most have a mean, variance, etc. Two random variables can be independent and we can condition a random variable on an event, or on the value of another random random variable. We will now explore some of these concepts further.

## L3.2 Distributions

The **distribution function** of a random variable $X$ is the function

$$F_X(x) = P(X \le x)$$

The distribution of a discrete random variables is characterized by its **probability mass function**, or pmf, defined by

$$p_X(x) = P(X=x)$$

For example, the pmf of the outcome of the throw of a single die is $p_X(x) = 1/6$.

Calculating the pmf of a discrete random variable is conceptually simple. Just follow the two steps:

1. Collect all possible outcomes that give rise to the event {X=x}
2. Add their probabilities to obtain $p_X(x)$

### Exercise: Find the pmf of the random variable defined as the number of heads when tossing a coin two times

Since it is guaranteed that some outcome will happen, we always have

$$\sum_x{p_X(x)} = 1$$

If we know the pmf of a random variable, then we can find the probability of any set of values, $S$ by using

$$P(X \in S) = \sum_{x \in S}{p_X(x)}$$

Let's now explore the distribution of some famous random variables.

The first one is the **Bernoulli Random Variable**. It can take on the value of 1 with probability $p$ and 0 with probability $1-p$. It can represent success, e.g., hitting the bullseye in darts, getting a head when tossing a coin, or many other practical situations where one experiment is made with a probability fo success $p$. The pmf is simply

$$p_X(x) = \left\{
	\begin{array}{ll}
		p  & \mbox{if } x = 1 \\
		1-p & \mbox{if } x = 0
	\end{array}
\right.$$

Then, we have the **Binomial Random Variable**. It represents the number of success of a Bernoulli variable, with parameter $p$ after $n$ tries. As we saw last time, the pmf is the binomial distribution:

$$p_X(k) = P(X=k) = \binom{n}{k}p^k(1-p)^{n-k}$$ 


The **Geometric Random Variable** has the geometric distribution. It represents the number of trials until the first success of a Bernoulli random variable, with parameter $p$:

$$p_X(k) = P(N=k) = (1-p)^{k-1}p$$ for $k=1,2, ...$

Note that the range of the binomial random variable is 0 to $n$, while the geometric random variable can take any postive integer as its value.

Another important discrete random variable is the **Poisson Random Variable**, which will be used when we will discuss Poisson processes later on. It is also an approximation to the binomial for very large $n$ and small $p$ (in which case, computing the $\binom{n}{k}$ might be challenging while using Poisson as an approximation is easy):

$$p_X(k) = e^{-\lambda}\frac{\lambda^k}{k!}$$ for $k=0,1,2, ...$

TRY LARGE VAL AS WELL TO SHOW OVERFLOW

In [1]:
#Find the probability of 5 success in 100 trials using both binomial and the poisson approximation
import numpy as np
import scipy.special

n=100
k = 5
p = 0.01

b = scipy.special.binom(n,k)*p**5*(1-p)**95

print("Actual probability: ", b)
lam = n*p

p = np.exp(-lam)*1/np.math.factorial(5)
print("Approximate probability: ", p)

Actual probability:  0.00289778712376148
Approximate probability:  0.0030656620097620196


How do we sample from a generic discrete random variable with finitely many values? One way is as follows. Computers can generate samples from a uniform random variable in the range [0,1]  (most modern computer languanges have an easy way to generate it). Then, we can split the the range [0,1] by the $p_X(x)$ values, since they sum up to 1, and then pick a value accordingly. Here is a sample python code:

In [18]:
#We could use np.random.choise as well, but this is just a demo on how one could do it directly
#More efficient to use modulus

import numpy as np

x = [1,2,3,4,5,6]
p_X = 6*[1/6]
N = 2000

def discrete_sampling(x, p_X, N):
    
    cumulative_p = np.cumsum(p_X)
    n = len(x)
    samples = []
    for i in range(N):
        i = 0
        r = np.random.uniform()
        while r >= cumulative_p[i] and i<n:
            i+=1
        samples.append(x[i])

    return samples

values, counts = np.unique(discrete_sampling(x, p_X, N), return_counts=True)
print(values)
print(counts/N)

#p_X = [1/2, 1/24, 1/24, 1/3, 1/24, 1/24]
test = np.random.choice(x, N, True, p_X)
values, counts = np.unique(test, return_counts=True)
print(values)
print(counts/N)

[1 2 3 4 5 6]
[0.163  0.1785 0.161  0.1615 0.168  0.168 ]
[1 2 3 4 5 6]
[0.17   0.157  0.169  0.157  0.1665 0.1805]


In practice, one is very often working with discrete distributions, even when the underlying models are continuous. We may generate a million samples and then work with the resulting distribution to measure its properties.


## L3.3 Functions of random variables

Let's consider a random variable $X$, representing the outcome of a roll of a fair die so that $P(X=k)=1/6$ for $k=1,..,6$. Now, let's define $Y = (X-3)^2$. Let's find the probability mass function of $Y$.

In general, when $X$ is a discrete random variable and $Y=g(X)$, then we can find the pmf of $Y$ by walking through all values $x$ of $X$, computing $g(x)$, assigning a probability $p_X(x)$ to the value $g(x)$. The same value for $g(x)$ might occur several times for different $x$ values, so we need to aggregate those probabilities. The pmf for $Y$ is given by

$$p_Y(y) = \sum_{\{x|g(x)=y\}}{p_X(x)}$$

## L3.4 Expectation, mean and variance

The distributions give the complete probabilistic information about a random variable, but they can be cumbersome to use. Often, people want to know some summarized values, e.g., a typical value from the distribution, and the dispersion of the distribution. When people want a single number (often a bad idea, see the book The Flaw of Averages by S. Savage), they are typically asking for the mean, or the expected value. 

The expected value has an interpretation in that if you repeatedly sample from a random variable (e.g., throw a die) and compute the average outcome, then that value will approach the expected value of the rando variable as the number of attempts increase. This is called the law of large numbers, i.e.,

$$\frac{X_1 + X_2 + ... + X_n}{n}$$

will approach $EX$ as $n$ increases. For discrete random variables, the expected value is defined as 

$$EX = \sum_x{p_X(x)x}$$

Note that this is just a single number for a random variable $X$, and there is nothing random about it.

### Exercise: In a roullete, when you bet \\$1 on black, you have 18/38 chance of winning $1 but you lose your \\$1 with probability 20/38. What is your expected earnings?

The variance is defined as

$$\text{var}(X) = E(X - (EX))^2$$

The E operator is really an integral in disguise, but we don't have to worry about that. Whatever you have under the E, you just have to go through all possible values, multiply with the probability of that value, and sum everything together, i.e., 

$$Eg(X) = \sum_x{p_X(x)g(x)}$$

In other words, we do not need to find the pmf of $g(X)$ in order to compute this expected value. Thus, the formula for the variance of a discrete random variable can be written

$$\text{var}(X) =\sum_x{p_X(x)(x - EX)^2}$$

Let's find the expected value and variance of a Bernoulli random variable with probability $p$.

The famous standard deviation, which measures the dispersion, or volatility/fluctuation of the random variable around its mean, is then defined by

$$\sigma_X = \sqrt{\text{var}(X)}$$

The standard deviation is often easier to work with than the variance, because it has the same unit as $X$.

Here are the main properties of the expected value and the variance - often handy to use.

### Expectation is linear

$$E[aX + b] = aE[X] + b$$

Example: if $E[X] = 2$ and $Y = 3X+1$, then $E[Y] = 7$.

Another example is the mean of the binomial distribution, but the binomial is really just a sum of Bernoulli variables, so if $X$ is binomial(n,p), then $E[X] = np$.

### Constants do not effect variance and scaling squares the variance

$$\text{var}(aX + b) = a^2\text{var}(X)$$

Another convenient way to calculate the variance is the following formula

$$\text{var}(X) = E[X^2] - (E[X])^2$$

### Example
If the weather is good, which happens with probability 0.6, Anna walks the 2 km to class at the speed of V = 5km/h, and otherwise she rides her bike at a speed of V = 20 km/h. What is the mean of the time T it takes Anna to get to class?


It is incorrect to compute the mean of the speed first, and then the mean of the time (however, this is often done in practice):

$$E \left[ \frac{2}{V} \right] \ne \frac{2}{E[V]}$$

Here is another example of the flaw of averages, where the average value i used inside another function causing a severe issue from the book of Flaw of Averages by Sam Savage.

In 1997, the National Weather Service forcasted that the Red River is expected to crest (max hight) at roughly 50 feet (high). They actually provided a distribution, but mean was 50 feet and people fixated on it. Consider the hypothetical situation where the distribution is 45 feet (50\% chance) and 55 feet (50\% chance). The average is 50 feet. The dikes were designed to withstand a 50 feet crest, and if the crest is at or lower than 50 feet, then there is no damage, but if the crest goes above, then there would be a \\$2bn damage. In this situation, computing first the average of the hight, and then employing that average value in the damage equation, would result in a damage of $0. However, the actual average damage is \\$1bn! This actually happened, and the there was a higher crest forcing about 50,000 people out of their homes.

## L3.5 Enthropy


Entropy measures the uncertainty or disorder of a distribution. For example, a uniform distribution, like the toss of a die, has a high uncertainty because all values can occur with equal probability. The uncertainty is lower in, e.g., the sum of two dice, where the probability of obtaining a 7 is higher than the probability of obtaining other values, so there is some level of certainty, or lower level of uncertainty. The same concept exists in thermodynamics, or statistical mechanics, which measures the disorder within a system (e.g., a system with high entropy can be in many different states), and in information theory where the value of new information can be measured as the drop in enthropy.

The enthropy for a discrete random variable $X$ is defined as

$$H = -\sum_x{p(x)log(p(x))}$$

### Exercise: Compute the enthropy of a Bernoulli random variable with $p=0.3$

It can be shown that the maximum enthropy, or disorder, is obtained when all the $p_X(x)$ are the same.

Enthropy is also used quite a bit in machine learning. A good example are the classification trees. The following description comes from Andrew Ng. The tree starts with a root node that presents a choice between examples that exhibit a particular feature or not, leading to two child nodes that contain examples with and without that feature. Each child poses yet another choice that leads to two more children, and so on. The process ends with any number of leaf nodes, each of which, mostly or wholly, contains examples of one class.

To grow, the tree must find the root decision. To choose, it considers all features and their values and chooses the one that maximizes the purity of the split (Optimal purity is defined as 100 percent of examples of one class going to a particular child node and none going to the other node.) This is where entropy comes to play, as it measures the purity of the split, or the disorder in the split. Splits are rarely 100 percent pure (enthropy is 0 for 100% pure splits) after just one decision and may never get there, so the process continues, producing level after level of child nodes, until purity doesn’t rise much by considering further features. At this point, the tree is fully trained.
At inference, a fresh example traverses the tree, which evaluates a different decision at each level from top to bottom. The example takes the label of the data contained by the leaf node it lands in.

Suppose, e.g., we want to classify customers into two classes, likely to churn (or drop our service) and not likely to churn (admittedly very simplified). Then, we collect data on churn together with lots of other attributes, such as demographics (where you live, gender, age, etc.), and data about the customer's business (like how often do they use our service, have they complained before, how long have they used our service, etc.). When we start looking at our data, we may see that 5% of our customers churn - this can then be seen as our base distribution. We then go through all of our data attributes to see which one reduces the enthropy most. E.g., when we look at complaints, we split the data into two groups, one for those who ahev complained, and one for those who have not complained. We then measure the enthropy in each group, and combine them together using the population proportions in each group. If this new enthopy is lower than the one we started with, then we have some information gain by splitting the data this way. The first step is to perform this operation on all data attributes and choose the attribute that gives is the highest information gain. Then, we repeat the process for each of the subgroups, etc. until we have a decent classification tree for the classes of interest.

INTRODUCE THE FIRST PROJECT.

## Homework problems

### Binomial distribution
1. What value of $p$ maximizes $P(X=k)$ when $X$ is a binomial random variable with $n$ trials?
This is actually an important question in statistics as it gives the maximum likelihood estimator of $p$ when we observe $k$ sucesses in $n$ trials.

### Function of random variables
2. Let the the random variable $X$ have the pmf $p_X(x) = 1/9$ for $x= -4,-3,-2, -1, 0, 1,2,3,4$.  Find the distribution of $Y=X^2$.

### Expected values and variance
3. Find the expected value and variance of the random variable $Y$ in the previous problem

### Entropy
4. Let $X$ ~ binomial(2,0.2) and $Y$ ~ bernoulli(0.3) Which variable rerpesents higher disorder (enthropy)?
